# `Document1:` SimpleTrap as an example

## 0. start

First of all, you have a trap model in fusion 360 project, for example, ./f3d/htrap_overhang_9x5mm.f3d, 

![avatar](./figs/origin.png)

This document helps you generate a pickle file(.pkl), which stores electric potential(or field) districution of one electrode at voltalge 1V and other elctrodes at voltalge 0V. With this pickle file(.pkl), you can analyze various properties of field distribution for arbitrary electrode voltage configuration, of which we concern about multipole expansion the most. The analysis of pickle file(.pkl) in Document2

## 1. define your electrodes

Firstly you need to color your input model file(.stl file) and edit color_map to define your electrodes.

A trap always divide into different part(different electrodes). Here we use color to identify different electrodes(STL file stores color for each mesh). This step assign a name for each color(i.e. each electrode).

### (1) color your model (in fusion 360)

In fusion 360, assign different color(we recommend to use ../../bemCol_lib/bemCol.adsklib) for different electrodes. Same color for two electrodes means they are shorted.

![avatar](./figs/colored.png)

( We define a set of standard color lib ../../../bemCol_lib/bemCol.adsklib for two reasons: 1. when exporting to .stl file, fusion 360 will compress 0-256 color(RGBA32) into 0-32 color(RGBA16), which means that some similar colors in fusion 360 will become same color after export to .stl file. Our standard color lib avoid this problem. 2.If you use .stl file exported from fusion 360 DIRECTLY, naming the electrodes would be very convinient. If you use other apps such as Inventor or Meshlab to generate .stl file, you can ignore this second reason because color encoding are quite different in different apps)

Then export to an stl file in fusion360 file>export tab and put the file in ./trapstl . (here we cut our trap into a smaller one before exporting, because distant electrode parts have little influence)

### (2) define color_map